In [1]:
## make sure you're connected to database (as of 10/12/17, is hosted on kanefsky)
from __future__ import division

import pymongo as pm
conn = pm.MongoClient(port=20809)

import numpy as np
import matplotlib
from matplotlib import pylab, mlab, pyplot
plt = pyplot
import seaborn as sns

import gridfs
from PIL import Image
import base64


import pandas as pd
import json

from IPython.core.pylabtools import figsize, getfigs

%matplotlib inline
from pylab import *
from numpy import *

# append to path
sys.path.append('./sketches')
from preprocessing_utils import *

In [2]:
# define directory where baseline sketches will be saved out
save_dir = 'baseline_sketches'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
# mongo globals
conn = pm.MongoClient(port=20809)
DBNAME = conn['during_car_furniture_morphsketch']
COLNAME = DBNAME['baseline_physical.files']
coll=COLNAME

In [4]:
# define database connections -- how many workers?
db = conn['during_car_furniture_morphsketch']['baseline_physical.files']
dbe = conn['end_car_furniture_morphsketch']['baseline_physical']
workers = dbe.find({'WorkerID': {'$exists': True, '$ne':''}}).distinct('WorkerID')                                                       
workers = [w for w in workers if len(workers)>0]
print len(workers)

146


In [5]:
### List of workers & images that are not up to snuff
# good workers
good_workers = ['A1UNJWWSAB1D59','A1GXFMAC759VRM','A2ADEPVGNNXNPA']
# bad workers
bad_workers = ['A1A3TGZ7DKJWRW','A1EAEZ7VIHKQ2','A1A70T3F5D0A6E']
# bad sketch_id's (if there were cross outs, or blanks)
bad_ids = ['561297c18094844cae9f0728','5612975c8094844cae9f0718','561299c18094844cae9f076e','5611e4f78094844cae9f04b8','5612f00d8094844cae9f0aae','5612ed678094844cae9f0a94','561474228094844cae9f1f54']

In [6]:
## Render out furniture sketches
import urllib, cStringIO
furniture = ['bedChair', 'bedTable', 'benchBed', 'chairBench','chairTable', 'tableBench']
fs = gridfs.GridFS(conn['during_car_furniture_morphsketch'], 'baseline_physical')

## endpoint morphnums
morphlines = db.find({'$and': [{'morphline':{'$exists': True}}, {'category':'furniture'}]}).distinct('morphline')
all_morphnums = db.find({'$and': [{'morphline':{'$exists': True}}, {'category':'furniture'}]}).distinct('morphnum')
morphnums = ['0','99']
viewpoints = db.find({'$and': [{'viewpoint':{'$exists': True}}, {'category':'furniture'}]}).distinct('viewpoint')


In [8]:
Obj = []
Pose = []
Rep = []
Path = []
for ml in morphlines:
    for mn in morphnums:
        for vp in viewpoints:
            records = db.find({'morphline':ml, 'morphnum':mn, 'viewpoint':vp}).sort('trialNum')
            print ml + ' ' + mn + ' ' + vp + ' ' + str(records.count())
            for i,r in enumerate(records):
                if r['wID'] not in bad_workers:
                    try:
                        imgData = r['imgData']
                        im = Image.open(cStringIO.StringIO(base64.b64decode(imgData)))
                        fig = plt.figure(figsize=(8,8))
                        p = plt.subplot(1,1,1)
                        plt.imshow(im)
                        k = p.get_xaxis().set_ticklabels([])
                        k = p.get_yaxis().set_ticklabels([])
                        k = p.get_xaxis().set_ticks([])
                        k = p.get_yaxis().set_ticks([])
                        for spine in plt.gca().spines.values():
                            spine.set_visible(False) 

                        obj_ind = get_object_index(ml,int(mn))
                        obj_name = get_object_name(2,obj_ind)
                        pathdir = os.path.join(save_dir)
                        pathfile = os.path.join(save_dir,'{}_{}_{}.png'.format(obj_name,vp,i))
                        fig.savefig(pathfile,bbox_inches='tight')  
                        print 'Saving to ' + pathfile
                        plt.close(fig)        
                        Obj.append(obj_name)
                        Pose.append(int(vp))
                        Rep.append(i)
                        Path.append(pathfile)
                    except:
                        print('Something went wrong with {} {} {}'.format(ml,mn,vp))


chairBench 0 30 5
chairBench 0 15 25
chairBench 0 25 29
chairBench 0 20 4
chairBench 0 0 4
chairBench 0 10 5
chairBench 0 35 28
chairBench 0 5 28
chairBench 99 30 5
chairBench 99 15 25
chairBench 99 25 28
chairBench 99 20 5
chairBench 99 0 6
chairBench 99 10 6
chairBench 99 35 30
chairBench 99 5 25
tableBench 0 30 5
tableBench 0 15 29
tableBench 0 25 26
tableBench 0 20 4
tableBench 0 0 5
tableBench 0 10 5
tableBench 0 35 25
tableBench 0 5 27
tableBench 99 30 8
tableBench 99 15 27
tableBench 99 25 26
tableBench 99 20 5
tableBench 99 0 5
tableBench 99 10 5
tableBench 99 35 29
tableBench 99 5 28
chairTable 0 30 5
chairTable 0 15 26
chairTable 0 25 26
chairTable 0 20 4
chairTable 0 0 6
chairTable 0 10 4
chairTable 0 35 27
chairTable 0 5 29
chairTable 99 30 5
chairTable 99 15 29
chairTable 99 25 29
chairTable 99 20 5
chairTable 99 0 5
chairTable 99 10 5
chairTable 99 35 27
chairTable 99 5 27
bedTable 0 30 6
bedTable 0 15 27
bedTable 0 25 29
bedTable 0 20 4
bedTable 0 0 5
bedTable 0 10 5
bed

In [9]:
## save out metadata file
df = pd.DataFrame([Obj,Pose,Rep,Path])
df = df.transpose()
df.columns = ['object','viewpoint','instance','path']
df.to_csv('baseline_sketches_metadata.csv')